<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/5B_BERT_%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>BERT Twitter 文書分類 </center>

###<center>参考サイト</center>

###<center>https://qiita.com/takubb/items/fd972f0ac3dba909c293</center>

1. 準備（ライブラリのインストール）
2. データセットの準備
3. データの前処理（BERTが受けとれる形式にデータを整形）
4. 学習済みモデルのロード
5. 訓練（Pre-trainedモデルのファインチューニング）
6. 検証

#0. 作業環境の準備

##0.1. google driveのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##0.2. カレントディレクトリへ移動

In [2]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/.shortcut-targets-by-id/1OWWQmFsr-0HoCVHJBaVtdmaZ4m43kkOk/研究_M2/src


##0.3. GPUへ接続

In [3]:
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

##0.4. 現在時刻の取得

In [4]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  #print(DATETIME_now)

  return DATETIME_now

##0.5. 分類モデルの性格指標を指定

この指標をもとに、2クラス分類を行う。

E or I なら
EI指標の分類器を作成する。

In [5]:
#TYPE_INDEX='EorI_E'
#TYPE_INDEX='NorS_N'
#TYPE_INDEX='ForT_F'
TYPE_INDEX='JorP_J'

##0.6. 学習するデータセットの指定

ツイッターの各投稿に、MBTIの16性格がラベリングされている。


In [6]:
#INPUT_CSV_v6='./Assets/Assets_Output/Tweet_Datasets/v6S1_forBERT_2021-10-01.csv'
#INPUT_CSV_v6='./Assets/Assets_Output/Tweet_Datasets/v6S2_forBERT_2021-10-03.csv'
#INPUT_CSV_v6='./Assets/Assets_Output/Tweet_Datasets/v6S3_forBERT_2021-10-03.csv'
#INPUT_CSV_v6='./Assets/Assets_Output/Tweet_Datasets/v6S4_forBERT_2021-10-15.csv'
INPUT_CSV_v6='./Assets/Assets_Output/Tweet_Datasets/v6S5_forBERT_2021-10-23.csv'

#INPUT_CSV_v6='./Assets/Assets_Output/Tweet_Datasets/v6S7_forBERT_2021-11-08_U10sentences.csv' #実質S6

#INPUT_CSV_v6='./Assets/Assets_Output/Tweet_Datasets/v6S7_forBERT_2021-11-09_U20R10000.csv'

##0.7 学習パラメータを指定

In [7]:
LEANING_RATE=2e-5
BATCH_SIZE=64
EPOCH=20

In [8]:
SEASONS='S5'
HIGHER = 10
RAFFLE=10000


#SWITCH_CONTINUE ='on'
SWITCH_CONTINUE ='off'

In [9]:
TERM='%s_L%s_B%s_E%s_H%s_R%s' % (SEASONS, LEANING_RATE, BATCH_SIZE, EPOCH, HIGHER, RAFFLE)

print(TERM)

S5_L2e-05_B64_E20_H10_R10000


#1. 準備 (ライブラリのインストール)

Hugging Face Library 'Transformer' のダウンロード

* transformers
* 最新のNLPモデルを利用できるライブラリ
* BERTの他にもGPT-2, RoBERTa, XLM, DistilBert, XLNet, T5, CTRL...などの最新モデルや100言語以上の学習済みモデル（Pre-trained model）を利用可能

In [10]:
!pip install transformers

!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

#参考サイト (https://qiita.com/takubb/items/fd972f0ac3dba909c293) では欠けているライブラリ
!pip install fugashi
!pip install ipadic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.9)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.16)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already instal

#2. データセットの準備

##2.1.準備したツイートデータをダウンロードする。

In [11]:
import pandas as pd
df=pd.read_csv('%s' % INPUT_CSV_v6, engine='python')

# データの確認
print(f'データサイズ： {df.shape}')

df[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].sample(10)

データサイズ： (10003, 12)


,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
786,@xxxvis @tia_ship10 (´・ω・`)💦,2017-06-20 23:17:40+00:00,INFP,I,N,F,P
2677,人が多い時を避けて帰省する策士でございます笑,2017-04-28 20:06:10+00:00,ESFP,E,S,F,P
9146,キズナアイとミライアカリとのじゃロリ狐娘おじさんが勝手に対立させられてて笑う。,2017-12-21 03:15:45+00:00,ISFP,I,S,F,P
8168,飯食って帰ろうかな…いや寝る時間減るやろバカタレ…,2017-12-17 02:25:12+00:00,ISFP,I,S,F,P
385,MGS3の話をしたら…… スネークマッチとマカロフとSAAが欲しくなってしまった今日この頃…...,2017-12-25 13:08:52+00:00,ISTJ,I,S,T,J
5648,許せない誤訳②～My Heart Will Go On,2017-10-09 11:26:00+00:00,INFJ,I,N,F,J
1402,個人的にスーパーロボット風味なゴルドリーオは嫌いじゃ無いけど、アルディラッドカンバーの玄人向...,2017-09-25 14:03:37+00:00,INFP,I,N,F,P
9524,ノースポート先週行ったけど原因は何なんだろう,2017-11-10 04:09:48+00:00,ISFJ,I,S,F,J
1240,"@5ayakachan_bot 年始は平日の方が休み取りやすそう9,10辺りとか",2017-12-23 14:52:54+00:00,INTJ,I,N,T,J
1289,スポーツ時事とか100m走9.98以外何があるんや？,2017-09-13 12:08:09+00:00,ISFP,I,S,F,P


In [12]:
df.dropna(how='any', axis=0, inplace=True)
print(f'データサイズ： {df.shape}')

データサイズ： (9999, 12)


##2.2. 性格指標のone-hot化

In [13]:
df=pd.get_dummies(df, columns=["label_EorI", "label_NorS", "label_ForT", "label_JorP"], sparse=True)
df[['text', 'tweet_time', 'MBTI_Type','label_EorI_E','label_NorS_N','label_ForT_F','label_JorP_J']]

,text,tweet_time,MBTI_Type,label_EorI_E,label_NorS_N,label_ForT_F,label_JorP_J
0,とても久しぶりな動画製作中... 覚えたことは早速使いたい感が編集TLに出てる,2017-11-25 15:53:33+00:00,INFJ,0,1,1,1
1,道中大破の撤退を減らすには艦隊司令部を置けばいいんだろうけどそうするとどう考えてもボスで優勢...,2017-12-04 14:38:25+00:00,ISTJ,0,0,0,1
2,#セレスティア・ルーデンベルク生誕祭2017 誕生日おめでとうございます！！ ドッヒャーは未...,2017-11-22 15:11:32+00:00,INFP,0,1,1,0
3,今回は無かったですけど、タイミング次第であるみたいです😁,2017-12-24 02:17:13+00:00,INFP,0,1,1,0
4,最後のアレで持ってかれたw直虎面白かった(そして泣いた)直政の元服における口上も最高でしたヽ...,2017-12-17 10:00:02+00:00,ISTP,0,0,0,0
...,...,...,...,...,...,...,...
9998,今日初めて大学生相手に研修をやりましたが、とてもしんどかった。,2017-02-20 07:50:57+00:00,ENTP,1,1,0,0
9999,HKT48の最高かよにハマってしまった。。。,2017-05-20 15:05:43+00:00,INTP,0,1,0,0
10000,キングハサンは本人のカード性能理解しないと難しいなぁ,2017-12-29 17:23:09+00:00,INTP,0,1,0,0
10001,まぁ僕は両方とNFSPBもハンコン買いますが(バイト代消滅不可避) あとgtsport はナ...,2017-10-19 12:50:19+00:00,INFP,0,1,1,0


In [14]:
sentences= df['text'].values #sentences = df.sentence.values
labels= df['label_%s' % TYPE_INDEX].values #labels = df.label.values

In [15]:
labels 

[1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...]
Fill: 0
IntIndex
Indices: array([   0,    1,    6, ..., 9991, 9993, 9998], dtype=int32)

In [16]:
print(len(sentences))

9999


#3. データの前処理（BERTが受けとれる形式にデータを整形）

BERTでは基本的に学習済みモデルを利用する為、そのモデルが読み込めるフォーマットにデータを変換する必要があります
具体的には、以下の4つ手続きが必要になります

1. BERT Tokenizerを用いて単語分割・IDへ変換
>* 学習済みモデルの作成時と同じtokenizer（形態素解析器）を利用する必要がある
>* 日本語ではMecabやJuman++を利用されることが多い

2. Special tokenの追加
>* 文章の最後に[SEP]という単語する
>* 文章のはじめに[CLS]という単語を追加する（分類問題に利用される）
3. 文章の長さの固定
>* BERTでは全ての文書の長さ（単語の数）を同じにする必要がある（1文章あたりの最大の単語数は512単語）
>* そこで、Padding/Truncatingを用いて固定長に変換する
>* Paddingとは、指定した長さに満たない文章を[Pad]という意味を持たない単語の埋める処理
>* Truncatingとは、指定した長さを超える単語を切り捨てること
4. Attention mask arrayの作成
>* [Padding]を0、それ以外のTokenを1とした配列
>* 一見ややこししそう見えますが、どれもTransformerのライブラリでサポートされているので、簡単に実行できます

また、以下のイメージも参考にモデルの全体像と照らし合わせながら前処理内容をみるとイメージが沸きやすいかもしれません

##3.1. BERT Tokenizerを用いて単語分割・IDへ変換

In [17]:
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

#念の為、確認
## テスト実行
# 元文章
print(' Original: ', sentences[0])
# Tokenizer
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
# Token-id
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  とても久しぶりな動画製作中... 　覚えたことは早速使いたい感が編集TLに出てる
Tokenized:  ['とても', '久', '##し', '##ぶり', 'な', '動画', '製作', '中', '...', '覚え', 'た', 'こと', 'は', '早速', '使い', 'たい', '感', 'が', '編集', 'T', '##L', 'に', '出', 'てる']
Token IDs:  [8567, 1658, 28454, 14657, 18, 4884, 1300, 51, 3215, 8806, 10, 45, 9, 21966, 3276, 1549, 832, 14, 2028, 260, 28743, 7, 71, 7134]


##3.2. 次に文章の長さを固定する為に、文章あたりの最大単語数を確認します

ここでは、以下の方法で最大単語数を確認していますが、最大単語数は決め打ちで設定することも可能です

In [18]:
# print(sentences[15806])

In [19]:
from tqdm.notebook import tqdm
import numpy as np

# 最大単語数の確認
max_len = []
# 1文づつ処理
len_sentences = len(sentences)
for sent in tqdm(sentences):
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
len_max=max(max_len)
#len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
#len_q75=int(len_q75)
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

  0%|          | 0/9999 [00:00<?, ?it/s]

最大単語数:  129
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


##3.3. Tokenizerと最大単語数の確認がとれたので、全ての文章に一括して処理を行います

tokenizer.encode_plusを利用するとまとめて処理が行えます

In [20]:
from tqdm.notebook import tqdm

input_ids = []
attention_masks = []

# 1文づつ処理
for sent in tqdm(sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = len_max,        # 文章の長さを固定（Padding/Trancatinating）#mayo
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

  0%|          | 0/9999 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


##labelをtensor型に変換

labelはpd.get_dummiesでone-hotにしてます

In [21]:
# tenosor型に変換
labels = torch.tensor(labels)

## BERTの前処理が正常に実行されたか確認

前処理は、

Special tokenの追加、文章の長さの固定、Attention mask arrayの作成

である。

In [22]:
# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  とても久しぶりな動画製作中... 　覚えたことは早速使いたい感が編集TLに出てる
Token IDs: tensor([    2,  8567,  1658, 28454, 14657,    18,  4884,  1300,    51,  3215,
         8806,    10,    45,     9, 21966,  3276,  1549,   832,    14,  2028,
          260, 28743,     7,    71,  7134,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
 

1つ目の文章の前処理結果です

元々、日本語テキストだった文章がID化されています

Token IDsの最初に単語IDはSpecial tokenの[CLS]を表し、後半の0埋めが[Pad]を示しています



```
Original:  旧式Macで禁断のパワーアップ！最新PCやソフトを一挙にチェック
Token IDs: tensor([    2, 18718,  8653,    12,  1763, 29135,     5, 20734,   679,  6215,
         3794,    49,  1604,    11, 24598,     7,  9398,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])
```



##3.5. データローダの適用とデータの分割

ここまでで、前処理が完了したので、90%を訓練データ、10％をテストデータに分割して、pytorchで学習を行う為に、データローダーへ変換しておきます

データローダーの説明は、pytorchの基本的な操作なので本記事では割愛します（簡単に言うと、データをバッチごとに分割して、学習を上手くやってくれるデータ型です）

In [23]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%地点のIDを取得
# train_size = int(0.9 * len(dataset))
# val_size = len(dataset) - train_size
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# データセットを分割
from sklearn.model_selection import train_test_split
train_dataset, val_dataset  = train_test_split(dataset, train_size=(0.7+0.1))
val_dataset, test_dataset  = train_test_split(val_dataset, train_size=(0.7+0.1))

print('訓練データ数：{}'.format(len(train_dataset)))
print('検証データ数:　{} '.format(len(val_dataset )))
print('テストデータ数:　{} '.format(len(test_dataset)))

# データローダーの作成
batch_size = BATCH_SIZE

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
test_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

訓練データ数：7999
検証データ数:　1599 
テストデータ数:　401 


#4. 学習済みモデルのロード

huggingface transformerのBERTファインチューニングでは以下のタスクをサポートしています
全てBERT Pre-trainedモデルをベースとして学習し、出力層のみそれぞれのタスクに適した構成になっています

* BertModel
* BertForPreTraining
*   BertForMaskedLM
*   BertForNextSentencePrediction
*   BertForSequenceClassification
*   BertForTokenClassification
* BertForQuestionAnswering

公式ドキュメント.

## BertForSequenceClassification

今回は2クラス分類問題なのでBertForSequenceClassificationを利用します

BertForSequenceClassificationは**「BERT学習済みモデルの最後の層に分類用のレイヤー追加したネットワーク構成」**です

データを入力することで、学習済みモデル全体と未学習部分の分類機レイヤーの学習が行われます

デフォルトでは、モデル全体のパラメーターがファインチューニングされますが、学習させるレイヤーの設定変更も可能です

モデルロード時にnum_labelsを変更することで多クラス分類問題へも対応できます

In [24]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

学習済みモデルのロードが完了しました

次にこのモデルを手元のデータに合うようにファインチューニングを行います

#5. 訓練（Pre-trainedモデルのファインチューニング）

モデルのファインチューニングは通常のニューラルネットワークの学習と同様で、損失関数と最適化手法（Optimizer）を指定して、学習ループ（任意のバッチサイズとエポックを指定）を回します

BERTでは損失関数はすでに定義されているので、ハイパーパラメーターとして、最適化手法とバッチサイズをエポック数だけ指定します

ここでは、論文（BERT paper, Appendix A.3）に従って、最適化関数はAdamW（Adam Weight Decay fix）を利用し、ハイパーパラメーターは下記のように設定します

Batch size: 32
Learning rate: 2e-5
Epochs: 4

https://qiita.com/takubb/items/fd972f0ac3dba909c293

In [25]:
# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=LEANING_RATE)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    count_batch=0
    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs.loss
        logits= outputs.logits
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
        count_batch+=1
    train_loss = train_loss / count_batch #https://yoshinashigoto-blog.herokuapp.com/detail/30/
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 訓練モードをオフ
    valid_loss = 0
    count_batch=0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():
                    outputs_valid = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
                    loss = outputs_valid.loss
                    logits= outputs_valid.logits
            valid_loss += loss.item()
            count_batch+=1
    valid_loss= valid_loss/count_batch
    return valid_loss

学習に必要な関数が定義できたので、学習を行う

In [26]:
import pandas as pd
import numpy as np

try:
  df_performance=pd.read_csv('./%s_%s.csv' % (TERM, TYPE_INDEX), header=0)
  print('./%s_%s.csv を 読み込みました。' % (TERM, TYPE_INDEX))
except:
  list_performance=[]
  df_performance = pd.DataFrame(index=range(EPOCH), columns=['train_loss', 'valid_loss'])
  df_performance.index.name="epoch"
  df_performance.fillna(np.nan, inplace=True)
  df_performance.to_csv('./%s_%s.csv' % (TERM, TYPE_INDEX))
  print('./%s_%s.csv を 作成しました。' % (TERM, TYPE_INDEX))

./S5_L2e-05_B64_E20_H10_R10000_JorP_J.csv を 読み込みました。


In [27]:
df_performance

,epoch,train_loss,valid_loss
0,0,NaN,NaN
1,1,NaN,NaN
2,2,NaN,NaN
3,3,NaN,NaN
4,4,NaN,NaN
5,5,NaN,NaN
6,6,NaN,NaN
7,7,NaN,NaN
8,8,NaN,NaN
9,9,NaN,NaN


In [ ]:
import time
start = time.time()

# 学習の実行
max_epoch = EPOCH
list_train_loss = []
list_valid_loss = []
epoch=0

#学習途中の状態を読み込む。
if SWITCH_CONTINUE=='on':
  print('continue previous training...')
  print()
  checkpoint = torch.load("./model_%s_%s.tar" % (TERM, TYPE_INDEX))
  model.load_state_dict(checkpoint["model_state_dict"])
  optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
  epoch = checkpoint["epoch"]
  list_train_loss=df_performance['train_loss'].values.tolist()
  list_valid_loss=df_performance['valid_loss'].values.tolist()
  SWITCH_CONTINUE = 'off'

for epoch in range(epoch, max_epoch):
    print("%s epoch" % (epoch))

    train_ = train(model)
    list_train_loss.append(train_)
    
    # 学習途中の状態を保存する。 #https://pystyle.info/pytorch-save-and-load-model/
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
         },
         "model_%s_%s.tar" % (TERM, TYPE_INDEX),
    )

    valid_ = validation(model)
    list_valid_loss.append(valid_)

    print(df_performance.columns)
    df_performance.loc[epoch]['train_loss']=list_train_loss[epoch]
    df_performance.loc[epoch]['valid_loss']=list_valid_loss[epoch]
    # df_performance['train_loss']=list_train_loss[epoch]
    # df_performance['valid_loss']=list_val_loss[epoch]
    df_performance.to_csv('./%s_%s.csv' % (TERM, TYPE_INDEX))
    print(' Added to ./%s_%s.csv.' % (TERM, TYPE_INDEX))

    elapsed_time = time.time() - start
    print(" %s epoch  :  %s [sec] → %s" % (epoch, elapsed_time, pull_Datetime()))
    print()
print("終了時間 : %s [sec] → %s" % (elapsed_time, pull_Datetime()))

0 epoch
Index(['epoch', 'train_loss', 'valid_loss'], dtype='object')
 Added to ./S5_L2e-05_B64_E20_H10_R10000_JorP_J.csv.
 0 epoch  :  334.094797372818 [sec] → 2021-11-11 01:33:41.497000+09:00

1 epoch


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Index(['epoch', 'train_loss', 'valid_loss'], dtype='object')
 Added to ./S5_L2e-05_B64_E20_H10_R10000_JorP_J.csv.
 1 epoch  :  667.7168579101562 [sec] → 2021-11-11 01:39:15.101179+09:00

2 epoch


In [ ]:
print('list_train_loss : ', list_train_loss)
print('list_valid_loss : ', list_valid_loss)

print(valid_)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot learning curve
plt.figure()
plt.plot(list(range(epoch+1)), list_train_loss, 'r-', label='train_loss')
plt.plot(list(range(epoch+1)), list_valid_loss, 'b-', label='val_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid()
plt.title('%s  %s' % (TERM, TYPE_INDEX))

plt.savefig('./loss_%s_%s.jpeg' % (TERM, TYPE_INDEX))

#6. 検証

In [ ]:
# 検証方法の確認（1バッチ分で計算ロジックに確認）

model.eval()# 訓練モードをオフ
try:
    for idx, batch in enumerate(test_dataloader):
        print(idx)
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():   
            # 学習済みモデルによる予測結果をpredsで取得     
            preds = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)
except:
    pass

以下の様な結果が出力されます
この確率の様な値（厳密には、この値ををソフトマックス関数に入力すると確率になる）が大きい方のラベルをモデルは分類結果とし予測としています

In [ ]:
## 予測結果の確認
print(f'出力:{preds}')



```
出力:(tensor([[-5.0226,  5.0193],
        [-5.0390,  4.9736],
        [ 4.7941, -4.7459],
        [-4.9395,  4.6827], device='cuda:0'),)
```



左からラベルの[0, 1]に対応しており、1つ目のデータであれば、右側の値が大きいのでモデルはラベル1を予測している

In [ ]:
# 比較しやすい様にpd.dataframeへ整形
import numpy as np
import pandas as pd
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

accuracy_df.to_csv('./%s.csv' % TYPE_INDEX)

In [ ]:
accuracy_df

In [ ]:
accuracy_df['answer'] = (accuracy_df['pred_label'] == accuracy_df['true_label'])
list_answer=accuracy_df['answer'].to_list()
count_true=list_answer.count(True)
print(list_answer)
print(count_true)
print('accuracy : ',  float(count_true / len(list_answer))*100)

In [ ]:
pred_df.values

上記の様に予測ラベルと正解ラベルを取得して、dataframeへ保存することができました

あとは、シンプルに正解率を計算するなり、sklearnを用いて混合行列を作成するなり、好きな様に分類モデルの評価を行うことができる。